<a href="https://colab.research.google.com/github/ddekun/NLP/blob/lesson4/lesson4/hw4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Введение в обработку естественного языка

### Урок 4. Тематическое моделирование. EM-алгоритм

продолжаем исследование датасета с твитами

Скачиваем датасет (источник): положительные, отрицательные.

рабочие ссылки с твитами https://disk.yandex.ru/i/v5HM-ENiGXZVpQ https://disk.yandex.ru/i/koR5eMCToCZS2Q

как альтернатива можно скачать данные из Роспотребнадзора https://zpp.rospotrebnadzor.ru/Forum/Appeals для этого берём ноутбук parse_rospotrebnadzor.ipynb устанавливаем количество скачанных страниц больше не 50-сят хотябы 500 и для анализа берём только вопросы так как ответы есть не всегда

что надо сделать

объединить в одну выборку (это только для твитов), для роспотребнадзора сформировать датасет из вопросов провести исследование и выявить тематики о которых говорят в твитах (для твитов), а для роспотребнадзора так же выявить тематики о которых люди пишут проанализировать сделать визуализацию кластеров тематик проинтерпритировать получившиеся тематики

In [ ]:
! pip install pymorphy2

In [ ]:
!pip install -U gensim

In [ ]:
!pip install pyLDAvis

In [ ]:
import pandas as pd
import numpy as np

import re
import nltk
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger_ru')
import pymorphy2

from nltk.corpus import stopwords
from tqdm.notebook import tqdm
from multiprocessing import Pool
from pymystem3 import Mystem
from gensim.models import *
from gensim import corpora
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [ ]:
positive = pd.read_csv('/content/positive.csv', sep=';', usecols=[3], names=['text'])
negative = pd.read_csv('/content/negative.csv', sep=';', usecols=[3], names=['text'])
data = positive.append(negative)

In [ ]:
data.head()

In [ ]:
morph = pymorphy2.MorphAnalyzer()

In [ ]:
words_regex = re.compile('\w+')

def find_words(text, regex = words_regex):
    tokens =  regex.findall(text.lower())
    return [w for w in tokens if w.isalpha() and len(w) >= 3]


stopwords_list = stopwords.words('russian')


add_stop_words = ['ахах', 'мочь', 'супер', 'хотяб', 'ахахаххах', 'аахха', 'настюха',
                  'оля', 'блин']
stopwords_list += add_stop_words



def lemmatize(words, lemmer = morph, stopwords = stopwords_list):
    lemmas = [lemmer.parse(w)[0].normal_form for w in words]
    return [w for w in lemmas if not w in stopwords
            and w.isalpha()]


def preprocess(text):
    result = (lemmatize(find_words(text)))
    return nltk.pos_tag(result, lang='rus')

In [ ]:
data.text.iloc[10]

In [ ]:
print(preprocess(data.text.iloc[10]))

In [ ]:
preprocessed_text = list(tqdm(map(preprocess, data.text), total=len(data)))

In [ ]:
preprocessed_text[:5]

In [ ]:
def only_nouns(preprocessed_text):
    token_list = []
    for tweet in preprocessed_text:
        token_list.append([w[0] for w in tweet if w[1]=='S'])
    return token_list

In [ ]:
data['text'] = only_nouns(preprocessed_text=preprocessed_text)
data.sample(5)

In [ ]:
data = data[data['text'].map(lambda d: len(d)) > 2]
data.head()

In [ ]:
dictionary = corpora.Dictionary(data['text'])

dictionary.filter_extremes(no_below = 10, no_above = 0.9, keep_n=None) # игнорируем слова, которые встречаются реже 10 раз или составляют более 0.9 словаря
dictionary.save('tweet.dict')

In [ ]:
corpus = [dictionary.doc2bow(text) for text in data['text']]
corpora.MmCorpus.serialize('tweet.model', corpus)

In [ ]:
%time

lda = ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=3, chunksize=200, update_every=1, passes=2)

In [ ]:
lda.show_topics(num_topics=5, num_words=10, formatted=True)